In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from TurkishStemmer import TurkishStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

In [2]:
df = pd.read_csv('assets/lyrics/lyrics_scraped_20180621-134937.csv', index_col = 0)

In [3]:
df.shape

(1326, 4)

In [4]:
df.head()

,artist,title,album,text
0,Sezen Aksu,1945,Git,1945 Gel asırlardan uzan da tut ellerimi sımsı...
1,Sezen Aksu,1980,1980,Sigaramın dumanına sarsam saklasam seni Gitme...
2,Sezen Aksu,Ablam Aşktan Öldü,şarkı söylemek lazım,Ablam aşktan öldü Herşey filmlerdeki gibi oldu...
3,Sezen Aksu,Acıtmışım Canını Sevdikçe,Öptüm,Hani yangın yerinde Sevmiştim ben seni Darmada...
4,Sezen Aksu,Ada Vapuru,Öptüm,ADA VARUPU YANDAN ÇARKLI BAYRAKLAR DONANMIŞ CA...


In [5]:
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    if pd.isnull(text):
        return []
    # tokenizing and removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    # removing any stopwords
    text_processed = [word.lower() for word in text_processed if word.lower() not in stopwords.words('turkish')]
    
    # stemming
    stemmer = TurkishStemmer()
    
    text_processed = [stemmer.stem(word) for word in text_processed]
    
    try:
        text_processed.remove('b')
    except: 
        pass

    return " ".join(text_processed) ## <-- we're keeping our words distinct

In [9]:
test_text = df['text'][34]

In [10]:
text_process(test_text)

'bir yer bir çocuk doğt sancı iç duyuyor bilir kim çocuk göz güneş görüyor bir yer bir çocuk doğt sancı iç duyuyor bilir kim çocuk göz güneş görüyor göz hayat pırıl pırıl eller ümit var dünya pislik sıkıl minik yumruk dünya pislik sıkıl minicik yumruk güzel bebek bura bura i̇nsan sevg kokuyor bir küçük dünya varlık sırt dünya taşıyor güzel bebek bura bura i̇nsan sevg kokuyor bir küçük dünya varlık sırt dünya taşıyor'

In [30]:
df['processed_text'] = df['text'].map(text_process)

In [34]:
df['processed_text']
df['num_words'] = [len(x) for x in df['text'].str.split(" ")]

In [42]:
df.loc[1,['text', 'num_words']]['num_words']

106

In [44]:
df.sort_values('num_words', ascending=False)

,artist,title,album,text,processed_text,num_words
1089,Zeki Müren,Uzun Yıllar Bekledim (Kakikat Oldu Rüyam),Büyük Bestekarlar Serisi Zeki Müren 4,ne zaman iki satir yazmaya kalksam hep sana h...,zaman ik satir yazma kalk san sen biz yaziyor ...,705
1238,Kazım Koyuncu,Oy Trabzon Trabzon,Ölümsüzlük Konseri,Oy Trabzon Trabzon oy hep dağlari sardunuz ...,oy trabzon trabzon oy dağlari sart 2x yüreğum ...,477
547,Tarkan,I Wanna Hear Love Speak,Yine Sensiz,"don't speak, you are beatiful don't say a wo...",don t speak yo are beatiful don t say a word a...,477
58,Sezen Aksu,Gelsin Hayat Bildiği Gibi (Ft. Ceza),Yürüyorum Düş Bahçeleri'nde,"Birer birer kayıp giderde her bir sevilen, yen...",birer birer kayıp gider bir sevilen yeni gelme...,417
129,Sezen Aksu,Seni Yerler,Düş Bahçeleri,Sen bizim mahalleye geldin geleli canım Bizd...,sen biz mahalle gelt gele can biz akıl kalt fi...,393
1306,Barış Manço,Nane Limon Kabuğu,Sahibinden İhtiyaçtan,Eski adamlar doğruyu söylemiş Bir çiçekle bah...,esk adam doğru söyl bir çiçek bahar olmaz kiş ...,366
570,Tarkan,Start The Fire,Come Closer,"Baby, I'm feeling your fire I'm feeling you c...",baby i m feeling your fir i m feeling yo close...,348
548,Tarkan,I'm Gonna Make U Feel Good,Come Closer,I'm not as innocent as I seem Focus on me bab...,i m not as innocent as i see focus on me baby ...,342
1205,Ahmet Kaya,Uy Havar,Sevgi Duvarı,Yangınlar Kahpe fakları Korku cığlıkları ...,yang kahpe fak kork cığlık ir sel aç yırtıç su...,339
1262,Barış Manço,Ayı,Çocuk Şarkıları Volume 1,Bugün hava güzel dedim ki hanıma Haydi kalk ...,bugü hav güzel det han haydi kalk giy çıkal bi...,325


In [48]:
test = df.loc[1246, 'text']

In [54]:
test

"Adem babayla Havva anadan bu yana   Çok şeyler söylendi sevda üzerine  Sayısız türküler yapıldı Leyla ile Mecnun Ferhat ile Şirin  Hepside dertli ayrılıklarla biten ızdırap gözyaşı dolu  Hani karabasan gibi insanın dünyasını karartan sıkıntılı şeylerdi  Bugün Barış kardeşiniz sizlere yeni bir türkü söyleyecek  Çünkü Barış gördü ki yeryüzünde en büyük gerçek  Adem oğlu kızgın fırın Havva kızı mercimek    Kız dediğin nazlı olur erkek ise mangal yürek  Er kişinin yanında hatun gerek  Kız dediğin nazlı olur erkek ise mangal yürek  Er kişinin yanında hatun gerek  Adem oğlu kızgın fırın Havva kızı mercimek    Kız olmazsa er kişi kolsuz kanatsız demek  Davulu dengi dengine vurmak gerek  Kız olmazsa er kişi kolsuz kanatsız demek  Davulu dengi dengine vurmak gerek  Adem oğlu kızgın fırın Havva kızı mercimek    eval(ez_write_tag([[336,280],'alternatifim_com-box-4','ezslot_2']));  [ reklamı gizle   hide ads ]  Adem oğlu kızgın fırın Havva kızı mercimek  Mercimek fırın yan yana fazla söze ne gere

In [55]:
pattern = r'(eval.*])'

In [57]:
test = re.sub(pattern, "", test)
test = re.sub(r"\s+", " ", test)
test

"Adem babayla Havva anadan bu yana Çok şeyler söylendi sevda üzerine Sayısız türküler yapıldı Leyla ile Mecnun Ferhat ile Şirin Hepside dertli ayrılıklarla biten ızdırap gözyaşı dolu Hani karabasan gibi insanın dünyasını karartan sıkıntılı şeylerdi Bugün Barış kardeşiniz sizlere yeni bir türkü söyleyecek Çünkü Barış gördü ki yeryüzünde en büyük gerçek Adem oğlu kızgın fırın Havva kızı mercimek Kız dediğin nazlı olur erkek ise mangal yürek Er kişinin yanında hatun gerek Kız dediğin nazlı olur erkek ise mangal yürek Er kişinin yanında hatun gerek Adem oğlu kızgın fırın Havva kızı mercimek Kız olmazsa er kişi kolsuz kanatsız demek Davulu dengi dengine vurmak gerek Kız olmazsa er kişi kolsuz kanatsız demek Davulu dengi dengine vurmak gerek Adem oğlu kızgın fırın Havva kızı mercimek Adem oğlu kızgın fırın Havva kızı mercimek Mercimek fırın yan yana fazla söze ne gerek Deli gönül sevdi mi istemez yorgan döşek Adem oğlu kızgın fırın Havva kızı mercimek Kimi zeytin peynir yer kimi baklava böre

In [86]:
def remove_ads(text):
    pattern = r'(eval.*])'
    return re.sub(pattern, "", text)

In [87]:
def regularize_whitespace(text):
    whitepattern = r'\s+'
    return re.sub(whitepattern, " ", text)

In [88]:
def remove_and_reg(text):
    text = remove_ads(text)
    text = regularize_whitespace(text)
    return text

In [89]:
df['fixed_text'] = df['text'].map(remove_and_reg)

In [90]:
df['fixed_text']

0       1945 Gel asırlardan uzan da tut ellerimi sımsı...
1       Sigaramın dumanına sarsam saklasam seni Gitme ...
2       Ablam aşktan öldü Herşey filmlerdeki gibi oldu...
3       Hani yangın yerinde Sevmiştim ben seni Darmada...
4       ADA VARUPU YANDAN ÇARKLI BAYRAKLAR DONANMIŞ CA...
5       Ben bu dünyaya bir türlü alışamadım Bu yüzden ...
6       Bölünür sancıyla uykular Sığınak değil en kuyt...
7       Bu şehrin meydanlarında, garında, rıhtımında S...
8       Aglamak guzeldir Suzulurken yaslar gozunden Sa...
9       Uzun uzun aynaya baktım Dedim ki kendime "Kız ...
10      Uzanıp Kanlıca'nın orta yerinde bir taşa Gözüm...
11      Titresin bir mum alevinde o eski günler Bir gü...
12      Geçti yıllar ah geç aydım Anladım ki boşa gün ...
13      Yar yolunu kolladım İpek mendil salladım Ona ç...
14      Batarken ufuktan bir akşam güneşi Bırakıp gitm...
15      Gece inerken Söner perde perde Grubun rengi De...
16      Ellerin ne kadar sıcak Tıpkı ateş gibi Bakışla...
17      Namlun

In [91]:
df[df['title'].str.find('Dost') > 0]

,artist,title,album,text,processed_text,num_words,fixed_text
290,İbrahim Tatlıses,İşte Dostlar,NaN,Kınamayın dostlar allah aşkına Yalvardım yakar...,kınamay dost allah aşk yalvar yakar dönmüyor g...,41,Kınamayın dostlar allah aşkına Yalvardım yakar...
936,Zeki Müren,Eski Dost Düşman Olmaz,Nostaljik Parçalar,Seviyorum demiştin hem de yeminle Söyle kalbin...,seviyor demiş yem söyl kalbi acep şimt kim sev...,62,Seviyorum demiştin hem de yeminle Söyle kalbin...
937,Zeki Müren,Eskimeyen Dost,Eskimeyen Dost,"Ne söylesen sen haklısın, aylar varki görülmed...",söyl sen hakl ay varki görül bir köşe oturup b...,103,"Ne söylesen sen haklısın, aylar varki görülmed..."
1287,Barış Manço,Geçti Dost Kervanı,Estağfurullah Ne Haddimize,Şu karşı yaylada göç katar katar Bir yiğidin d...,karş yayla göç katar katar bir yiği dert ser t...,106,Şu karşı yaylada göç katar katar Bir yiğidin d...


In [92]:
df.loc[1287, 'fixed_text'].strip()

'Şu karşı yaylada göç katar katar Bir yiğidin derdi serinde tüter Bu ayrılık bana ölümden beter Geçti dost kervanı eyleme beni eyleme beni Bu ayrılık bana ölümden beter Geçti dost kervanı eyleme beni eyleme beni Şu benim sevdiğim başta oturur Bu güzelin derdi beni bitirir Bu ayrılık bize zulüm getirir Geçti dost kervanı eyleme beni eyleme beni Bu ayrılık bize zulüm getirir Geçti dost kervanı eyleme beni eyleme beni Pir Sultan Abdalım dağlar aşarım Aşarımda dost diye nere düşerim Çok ekmeğin yedim helalaşalım Geçti dost kervanı eyleme beni eyleme beni Çok ekmeğin yedim helalaşalım Geçti dost kervanı eyleme beni eyleme beni'

In [70]:
df.loc[1287, 'text']

'Şu karşı yaylada göç katar katar Bir yiğidin derdi serinde tüter Bu ayrılık bana ölümden beter Geçti dost kervanı eyleme beni eyleme beni Bu ayrılık bana ölümden beter Geçti dost kervanı eyleme beni eyleme beni  Şu benim sevdiğim başta oturur Bu güzelin derdi beni bitirir Bu ayrılık bize zulüm getirir Geçti dost kervanı eyleme beni eyleme beni Bu ayrılık bize zulüm getirir Geçti dost kervanı eyleme beni eyleme beni  Pir Sultan Abdalım dağlar aşarım Aşarımda dost diye nere düşerim Çok ekmeğin yedim helalaşalım Geçti dost kervanı eyleme beni eyleme beni Çok ekmeğin yedim helalaşalım Geçti dost kervanı eyleme beni eyleme beni   '

In [78]:
df[df['text'].str.contains(" les")]['text']

407    Faisons l'amour avant de nous dire adieu Faiso...
410    Que je doıs te rencontrer  Et que je doıs t'ad...
429    Bilsen neler dönüyor şu garip dünyada  Arkadaş...
430    Il me semble bien que nous allons nous connaît...
434    İstanbul, istanbul  İstanbul c'est constantıno...
449    Git ona git benden selam söyle selam söyle Ara...
452    Je n'aı jamaıs su dıre  Les mots qu'ıl fallaıt...
Name: text, dtype: object

In [83]:
df.loc[449, 'fixed_text'].strip()

"Git ona git benden selam söyle selam söyle Aramasın artık hiç beni öyle beni öyle Git ona git benden selam söyle selam söyle Son pişmanlık fayda etmez git ona söyle  Git ona git benden selam söyle selam söyle Aramasın artık hiç beni öyle beni öyle Şimdi çok mutluyum yanımdaki sevgilimle Son pişmanlık fayda etmez git ona söyle  Onun burnu kaf dağında Söz söylenmez yanında Kendini beğenmişin biri o  Git ona git benden selam söyle selam söyle Nispet yapmasın hiç bana öyle bana öyle Git ona git benden selam söyle selam söyle Unutamaz beni bin yıl geçse git ona söyle  Pourquoı parler d'amour Quand on s'aıme quand on s'aıme Nos cœurs depuıs toujours Sont les memes sont les memes  Pas besoın de dıscours Pas la peıne de poeme Les mots d'amour rendront servıce Pour te les dıre aussı Regarde-moı, je suıs heureuse"

In [84]:
df.loc[449, 'text'].strip()

"Git ona git benden selam söyle selam söyle Aramasın artık hiç beni öyle beni öyle Git ona git benden selam söyle selam söyle Son pişmanlık fayda etmez git ona söyle  Git ona git benden selam söyle selam söyle Aramasın artık hiç beni öyle beni öyle Şimdi çok mutluyum yanımdaki sevgilimle Son pişmanlık fayda etmez git ona söyle  Onun burnu kaf dağında Söz söylenmez yanında Kendini beğenmişin biri o  Git ona git benden selam söyle selam söyle Nispet yapmasın hiç bana öyle bana öyle Git ona git benden selam söyle selam söyle Unutamaz beni bin yıl geçse git ona söyle  Pourquoı parler d'amour Quand on s'aıme quand on s'aıme Nos cœurs depuıs toujours Sont les memes sont les memes  Pas besoın de dıscours Pas la peıne de poeme Les mots d'amour rendront servıce Pour te les dıre aussı Regarde-moı, je suıs heureuse"

In [102]:
df[(df['artist'] == 'Candan Erçetin') & df['album'].str.contains("Aranjman")]

,artist,title,album,text,processed_text,num_words,fixed_text
410,Candan Erçetin,Bak Bir Varmış Bir Yokmuş (C'est Ecrit Dans Le...,Aranjman,Que je doıs te rencontrer Et que je doıs t'ad...,que je doıs te rencontrer et que je doıs t ado...,158,Que je doıs te rencontrer Et que je doıs t'ado...
427,Candan Erçetin,Her Yerde Kar Var (Tombe La Neige ),Aranjman,Tombe la neıge Tu ne vıendras pas ce soır Tomb...,tombe la neıge tu vıendras pas ce soır tombe l...,56,Tombe la neıge Tu ne vıendras pas ce soır Tomb...
429,Candan Erçetin,Hoşgörsen (On S'embrasse Et On Oublie),Aranjman,Bilsen neler dönüyor şu garip dünyada Arkadaş...,bil ne dönüyor garip dünya arkadaşlık düşmanlı...,201,Bilsen neler dönüyor şu garip dünyada Arkadaşl...
434,Candan Erçetin,İstanbul,Aranjman,"İstanbul, istanbul İstanbul c'est constantıno...",i̇stanbul istanbul i̇stanbul c est constantıno...,137,"İstanbul, istanbul İstanbul c'est constantınop..."
440,Candan Erçetin,Karlar Düşer (Car Je Veux),Aranjman,İnşallah ol sen de böyle Aşık ol da bak birine...,i̇nşallah ol sen böyl aşık ol bak bir ben olt ...,66,İnşallah ol sen de böyle Aşık ol da bak birine...
449,Candan Erçetin,Selam Söyle (Pourquoi Parler D'amour),Aranjman,Git ona git benden selam söyle selam söyle Ara...,git on git ben selam söyl selam söyl ara ar be...,151,Git ona git benden selam söyle selam söyle Ara...
452,Candan Erçetin,Sessiz Gemi (Sans Toi Je Suis Seul),Aranjman,Je n'aı jamaıs su dıre Les mots qu'ıl fallaıt...,je n a jamaıs su dıre les mots qu ıl fallaıt p...,184,Je n'aı jamaıs su dıre Les mots qu'ıl fallaıt ...
